In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!ls "/content/drive/MyDrive/Facia_Expression/data/image"

origin.7z.001  origin.7z.003  origin.7z.005  origin.7z.007
origin.7z.002  origin.7z.004  origin.7z.006  origin.7z.008


In [ ]:
#!7za -y x "/content/drive/MyDrive/Facia_Expression/data/image/origin.7z*"

Output hidden; open in https://colab.research.google.com to view.

In [ ]:
#!cp -r ./origin "/content/drive/MyDrive/Facia_Expression"

In [ ]:
label_path = r"/content/drive/MyDrive/Facia_Expression/data/label/label.lst"
images_path=r"/content/drive/MyDrive/Facia_Expression/origin"

In [ ]:
import pandas as pd

face_data = pd.read_csv(label_path, sep=" " , header= None)
face_data

,0,1,2,3,4,5,6,7
0,angry_actor_104.jpg,0,28,113,226,141,22.9362,0
1,angry_actor_109.jpg,0,31,157,345,219,50.3056,0
2,angry_actor_120.jpg,1,53,53,372,372,13.9434,2
3,angry_actor_13.jpg,0,77,51,362,388,85.8104,3
4,angry_actor_132.jpg,0,95,31,412,476,82.3948,0
...,...,...,...,...,...,...,...,...
91788,surprised_expression_546.jpg,0,70,70,351,351,37.7117,5
91789,surprised_expression_381.jpg,0,51,61,117,107,91.6307,5
91790,surprised_expression_395.jpg,0,27,95,258,190,96.2861,5
91791,ecstatic_asian_31.jpg,0,60,136,184,108,39.9223,3


In [ ]:
face_data.columns =['image_name', 'face_id_in_image', 'face_box_top', 'face_box_left' , 'face_box_right' , 'face_box_bottom' , 'face_box_cofidence' , 'expression_label']
face_data

,image_name,face_id_in_image,face_box_top,face_box_left,face_box_right,face_box_bottom,face_box_cofidence,expression_label
0,angry_actor_104.jpg,0,28,113,226,141,22.9362,0
1,angry_actor_109.jpg,0,31,157,345,219,50.3056,0
2,angry_actor_120.jpg,1,53,53,372,372,13.9434,2
3,angry_actor_13.jpg,0,77,51,362,388,85.8104,3
4,angry_actor_132.jpg,0,95,31,412,476,82.3948,0
...,...,...,...,...,...,...,...,...
91788,surprised_expression_546.jpg,0,70,70,351,351,37.7117,5
91789,surprised_expression_381.jpg,0,51,61,117,107,91.6307,5
91790,surprised_expression_395.jpg,0,27,95,258,190,96.2861,5
91791,ecstatic_asian_31.jpg,0,60,136,184,108,39.9223,3


In [ ]:
face_data.shape

(91793, 8)

In [ ]:
face_cofidence = face_data[face_data.face_box_cofidence > 55]

In [ ]:
face_cofidence.shape

(39599, 8)

In [ ]:
from numpy.core.fromnumeric import resize
import cv2
import matplotlib.pyplot as plt
import os


stored_image = []
stored_label = []

for i, row in face_cofidence.sample(10000).iterrows():
    image_name = row["image_name"]
    left = row["face_box_left"]
    right = row["face_box_right"]
    top = row["face_box_top"]
    bottom = row["face_box_bottom"]
    label = row["expression_label"]
    img_path = os.path.join(images_path, image_name)
    read_image = cv2.imread(img_path)

    if read_image is not None:
         crop_image = read_image[top:bottom , left:right]
    else:
        continue

    if crop_image  is not None:
     crop_resize = cv2.resize(crop_image  , ( 64,64))

    else:
      continue

    stored_image.append(crop_resize)
    stored_label.append(label)



In [ ]:

import numpy as np

X = np.array(stored_image)
Y = np.array(stored_label)

In [ ]:
x = X / 255.0

In [ ]:
X_shaped = x.reshape((-1,64*64*3))
X_shaped.shape

(8393, 12288)

In [ ]:
import pandas as pd
pd.Series(Y).value_counts()

,count
3,3650
6,2513
4,771
5,616
1,396
0,362
2,85


In [ ]:
from imblearn.over_sampling import SMOTE

smote = SMOTE()
X_smote,Y_smote = smote.fit_resample(X_shaped, Y)

In [ ]:
pd.Series(Y_smote).value_counts()

,count
5,3650
3,3650
4,3650
0,3650
6,3650
1,3650
2,3650


In [ ]:
x = X_smote.reshape(-1,64,64,3)

In [ ]:
Y = Y_smote

In [ ]:
len(x)


25550

In [ ]:
len(X)

8393

In [ ]:
import numpy as np

from keras.utils import to_categorical
y = to_categorical(Y)

In [ ]:
import numpy as np
from sklearn.model_selection import train_test_split

X_train, X_temp, Y_train, Y_temp = train_test_split(x, y, test_size=0.3, random_state=42)
X_val, X_test, Y_val, Y_test = train_test_split(X_temp, Y_temp, test_size=0.5, random_state=42)


In [ ]:
import numpy as np
from tensorflow.keras.preprocessing.image import ImageDataGenerator
#from keras.preprocessing.image import ImageDataGenerator


img_datagen = ImageDataGenerator(

    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest'
)


img_datagen.fit(X_train)

## ***MODEL***

In [ ]:
import numpy as np
import tensorflow as tf
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, Flatten, Dense

model = Sequential()
model.add(Conv2D(32, (3, 3), activation='relu', input_shape=(64,64,3)))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Flatten())
model.add(Dense(128, activation='relu'))
num_classes = 7
model.add(Dense(num_classes, activation='softmax'))


/usr/local/lib/python3.10/dist-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [ ]:
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

history = model.fit(img_datagen.flow(X_train, Y_train, batch_size=32), epochs=50, validation_data=(X_val, Y_val))

Epoch 1/50


/usr/local/lib/python3.10/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


559/559 ━━━━━━━━━━━━━━━━━━━━ 39s 58ms/step - accuracy: 0.1833 - loss: 1.9225 - val_accuracy: 0.2539 - val_loss: 1.8250
Epoch 2/50
559/559 ━━━━━━━━━━━━━━━━━━━━ 28s 49ms/step - accuracy: 0.2396 - loss: 1.8490 - val_accuracy: 0.2610 - val_loss: 1.7946
Epoch 3/50
559/559 ━━━━━━━━━━━━━━━━━━━━ 27s 48ms/step - accuracy: 0.2651 - loss: 1.8155 - val_accuracy: 0.3116 - val_loss: 1.7109
Epoch 4/50
559/559 ━━━━━━━━━━━━━━━━━━━━ 28s 49ms/step - accuracy: 0.2882 - loss: 1.7719 - val_accuracy: 0.3178 - val_loss: 1.7000
Epoch 5/50
559/559 ━━━━━━━━━━━━━━━━━━━━ 28s 49ms/step - accuracy: 0.3142 - loss: 1.7214 - val_accuracy: 0.3627 - val_loss: 1.6131
Epoch 6/50
559/559 ━━━━━━━━━━━━━━━━━━━━ 45s 56ms/step - accuracy: 0.3425 - loss: 1.6694 - val_accuracy: 0.3870 - val_loss: 1.5651
Epoch 7/50
559/559 ━━━━━━━━━━━━━━━━━━━━ 29s 50ms/step - accuracy: 0.3701 - loss: 1.6259 - val_accuracy: 0.4298 - val_loss: 1.4766
Epoch 8/50
559/559 ━━━━━━━━━━━━━━━━━━━━ 40s 49ms/step - accuracy: 0.3850 - loss: 1.5906 - val_accurac

In [ ]:
loss ,accuracy =model.evaluate(X_test,Y_test )
print(f"Test Loss: {loss:.4f}")
print(f"Test Accuracy: {accuracy:.4f}")

120/120 ━━━━━━━━━━━━━━━━━━━━ 1s 7ms/step - accuracy: 0.6768 - loss: 0.8927
Test Loss: 0.8890
Test Accuracy: 0.6749


In [ ]:
model.save('/content/drive/MyDrive/My Projects/Expression Classification from Facial Images/facial_expression_model.h5')


In [ ]:
import cv2
import numpy as np

# Function to preprocess a single image
def preprocess_image(image_path):
    # Load the image
    image = cv2.imread(image_path)

    # Resize to match the model input
    image = cv2.resize(image, (64, 64))

    # Normalize pixel values to be between 0 and 1
    image = image / 255.0

    # Reshape to fit the model input shape
    image = np.reshape(image, (1, 64, 64, 3))

    return image

# Load the image you want to test
test_image_path = "/content/test.jpg"
test_image = preprocess_image(test_image_path)

# Use the trained model to make a prediction
prediction = model.predict(test_image)

# Get the class with the highest probability
predicted_class = np.argmax(prediction, axis=1)

# Print the result
print("Predicted Class:", predicted_class)


1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 1s/step
Predicted Class: [6]
